In [1]:
import numpy as np
import pandas as pd
import urllib.request # import the library we use to open URLs
from bs4 import BeautifulSoup # import the BeautifulSoup library so we can parse HTML and XML documents
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip -q install folium
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # specify which URL/web page we are going to be scraping
page=urllib.request.urlopen(url) # open the url using urllib.request and put the HTML into the page variable
soup= BeautifulSoup(page, "lxml")# parse the HTML from our URL into the BeautifulSoup parse tree format

In [3]:
print(soup.prettify()) #print results of the web page scrape

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"7d7f7700-9037-4d5c-b82b-6c37b4edbcf9","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":975466835,"wgRevisionId":975466835,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communicati

In [4]:
soup.title #Bring back page 'title' and data between the start and end of 'title' tags
soup.title.string #Bring back just titles that are strings

'List of postal codes of Canada: M - Wikipedia'

In [5]:
all_tables=soup.find_all("table")# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable

In [6]:
right_table=soup.find('table', class_='wikitable sortable') #place the right table "wikitable sortable" with all the needed information into right_table

In [7]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

In [8]:
dfToronto=df[~df.Borough.str.contains("Not assigned")]
dfToronto.drop_duplicates()
dfToronto.groupby(['Postal Code','Borough'], sort = False).agg(lambda x: ','.join(x))


,,Neighborhood
Postal Code,Borough,
M3A,North York,Parkwoods\n
M4A,North York,Victoria Village\n
M5A,Downtown Toronto,"Regent Park, Harbourfront\n"
M6A,North York,"Lawrence Manor, Lawrence Heights\n"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government\n"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village\n"
M1B,Scarborough,"Malvern, Rouge\n"
M3B,North York,Don Mills\n
M4B,East York,"Parkview Hill, Woodbine Gardens\n"


In [9]:
#question 2 of Week 3 Applied Data Science - Segmenting and Clustering Neighborhoods in Toronto
#Add locations to list of Postal Codes, Boroughs, and Neighborhoods
dfGeoloc = pd.read_csv ('http://cocl.us/Geospatial_data')

In [10]:
dfToronto = dfToronto.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
dfToronto['Borough']=df['Borough'].str.join('').str.replace(r"\n", "")
dfToronto['Neighborhood']=df['Neighborhood'].str.join('').str.replace(r"\n", "")

In [11]:
dfMerge=pd.concat([dfToronto, dfGeoloc],axis=1,join="inner")
dfMerge.head()

,Postal Code,Borough,Neighborhood,Postal Code,Latitude,Longitude
2,M3A,North York,Parkwoods,M1E,43.763573,-79.188711
3,M4A,North York,Victoria Village,M1G,43.770992,-79.216917
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1H,43.773136,-79.239476
5,M6A,North York,"Lawrence Manor, Lawrence Heights",M1J,43.744734,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1K,43.727929,-79.262029


In [12]:
df = dfMerge.loc[:,~dfMerge.columns.duplicated()]

In [13]:
#question 3 of Week 3 Applied Data Science - Segmenting and Clustering Neighborhoods in Toronto
#Map generation and exploration
#using Geopy library to get the latitude and longitude values of Toronto
#name agent tn_explorer
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to the map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='9c31cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [15]:
DT_Toronto = df['Borough']=='Downtown Toronto'#View and pick a neighborhood of interest by first filtering by Borough = Downtown Toronto
#Filter rows for Downtown Toronto
DT_YYZ=df[DT_Toronto]
DT_YYZ["Neighborhood"].unique()
#Selecting Harbourfront as the neighborhood of interest
neighborhood_latitude=43.773136
neighborhood_longitude=-79.239476

In [16]:
#FourSquare credentials
CLIENT_ID = 'GTNJYPNBOTG4LXIQT3KPNWIQEGBPT5SJYPIIVB0ZGZIYBKBT' # Foursquare ID
CLIENT_SECRET = 'HG54T0VI3CD2ZTP5KD45UXNCMU2VIOP5RH1YH02FE2QX5W1V' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GTNJYPNBOTG4LXIQT3KPNWIQEGBPT5SJYPIIVB0ZGZIYBKBT
CLIENT_SECRET:HG54T0VI3CD2ZTP5KD45UXNCMU2VIOP5RH1YH02FE2QX5W1V


In [17]:
#Explore Downtown Toronto - Harbourfront
#Get the top 100 venues in Harbourfront
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

#Send the GET request to examine the results
results = requests.get(url).json()

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
1,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
2,Thai One On,Thai Restaurant,43.774468,-79.241268
3,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500
4,TD Canada Trust,Bank,43.774830,-79.241251


In [21]:
nearby_venues.categories.unique()

array(['Hakka Restaurant', 'Caribbean Restaurant', 'Thai Restaurant',
       'Athletics & Sports', 'Bank', 'Bakery', 'Gas Station',
       'Fried Chicken Joint'], dtype=object)

In [23]:
nearby_venues["categories"].value_counts()

Bank                    1
Caribbean Restaurant    1
Fried Chicken Joint     1
Bakery                  1
Gas Station             1
Athletics & Sports      1
Thai Restaurant         1
Hakka Restaurant        1
Name: categories, dtype: int64